In [23]:
# 0
class Morph:
    def __init__(self, morph):
        surface, attr = morph.split("\t")
        attr = attr.split(",")
        self.surface = surface
        self.base = attr[6]
        self.pos = attr[0]
        self.pos1 = attr[1]

In [24]:
results = []
sentence = []

with open("ai.ja.txt.parsed", "r") as f:
    for line in f.readlines():
        if line[0] == "*":
            continue
        elif line != "EOS\n":
            sentence.append(Morph(line))
        else:
            results.append(sentence)
            sentence = []

for morph in results[2]:
    print(vars(morph))


{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}
{'surface': '（', 'base': '（', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'じん', 'base': 'じん', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'こうち', 'base': 'こうち', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'のう', 'base': 'のう', 'pos': '助詞', 'pos1': '終助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': 'AI', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'}
{'surface': '〈', 'base': '〈', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'エーアイ', 'base': '*\n', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '〉', 'base': '〉', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': '）', 'base': '）', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': 'と', 'base': 'と', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '「', 'base': '「', 'pos': '記号', 'pos1': '括弧開'}
{

In [25]:
# 1
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []

class Sentence:
    def __init__(self, chunks):
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks):
            if chunk.dst is not None:
                self.chunks[chunk.dst].srcs.append(i)

In [26]:
sentences, chunks, morphs = [], [], []

with open("ai.ja.txt.parsed", "r") as f:
    for line in f.readlines():
        if line[0] == "*":
            dst = int(line.split(" ")[2].rstrip("D"))
            if len(morphs) > 0:
                chunks.append(Chunk(morphs, dst))
                morphs = []
        elif line != "EOS\n":
            morphs.append(Morph(line))
        else:
            chunks.append(Chunk(morphs, dst))
            sentences.append(Sentence(chunks))
            morphs, chunks = [], []
            dst = None

for chunk in sentences[2].chunks:
    print([morph.surface for morph in chunk.morphs], chunk.dst, chunk.srcs)

['人工', '知能'] 17 []
['（', 'じん', 'こうち', 'のう', '、', '、'] 3 []
['AI'] 17 []
['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'] 5 [1]
['「', '『', '計算'] 9 []
['（', '）', '』', 'という'] 9 [3]
['概念', 'と'] 8 []
['『', 'コンピュータ'] 9 []
['（', '）', '』', 'という'] 10 [6]
['道具', 'を'] 12 [4, 5, 7]
['用い', 'て'] 12 [8]
['『', '知能', '』', 'を'] 13 []
['研究', 'する'] 14 [9, 10]
['計算', '機', '科学'] 15 [11]
['（', '）', 'の'] 16 [12]
['一', '分野', '」', 'を'] 17 [13]
['指す'] 34 [14]
['語', '。'] 20 [0, 2, 15]
['「', '言語', 'の'] 20 []
['理解', 'や'] 21 []
['推論', '、'] 22 [17, 18]
['問題', '解決', 'など', 'の'] 24 [19]
['知的', '行動', 'を'] 24 [20]
['人間', 'に'] 26 []
['代わっ', 'て'] 26 [21, 22]
['コンピューター', 'に'] 27 []
['行わ', 'せる'] 34 [23, 24]
['技術', '」', '、', 'または', '、'] 29 [25]
['「', '計算', '機'] 31 []
['（', 'コンピュータ', '）', 'による'] 31 [27]
['知的', 'な'] 33 []
['情報処理', 'システム', 'の'] 33 [28, 29]
['設計', 'や'] 34 []
['実現', 'に関する'] 35 [30, 31]
['研究', '分野', '」', 'と', 'も'] -1 [16, 26, 32]
['さ', 'れる', '。'] -1 [33, 34, 35]


In [43]:
# 2
for sentence in sentences:
    for chunk in sentence.chunks:
        if chunk.dst is not None:
            src = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
            dst = "".join([morph.surface if morph.pos != "記号" else "" for morph in sentence.chunks[chunk.dst].morphs])
            print(src, dst, sep="\t")

人工知能	人工知能
人工知能	語
じんこうちのう	エーアイとは
AI	語
エーアイとは	という
計算	道具を
という	道具を
概念と	という
コンピュータ	道具を
という	用いて
道具を	研究する
用いて	研究する
知能を	計算機科学
研究する	の
計算機科学	一分野を
の	指す
一分野を	語
指す	研究分野とも
語	推論
言語の	推論
理解や	問題解決などの
推論	知的行動を
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
代わって	行わせる
コンピューターに	技術または
行わせる	研究分野とも
技術または	コンピュータによる
計算機	情報処理システムの
コンピュータによる	情報処理システムの
知的な	実現に関する
情報処理システムの	実現に関する
設計や	研究分野とも
実現に関する	される
研究分野とも	される
される	される
日本大百科全書(ニッポニカ)』の	述べている
解説で	佐藤理史は
情報工学者通信工学者の	述べている
佐藤理史は	述べている
次のように	述べている
述べている	述べている
人間の	実現する
知的能力を	実現する
コンピュータ上で	技術ソフトウェアコンピュータシステム
実現する	技術ソフトウェアコンピュータシステム
様々な	ある
技術ソフトウェアコンピュータシステム	ある
応用例は	機械翻訳かな漢字変換構文解析等
自然言語処理	専門家の
機械翻訳かな漢字変換構文解析等	推論判断を
専門家の	模倣する
推論判断を	エキスパートシステム
模倣する	画像認識等が
エキスパートシステム	解析して
画像データを	検出抽出したりする
解析して	パターンを
特定の	検出抽出したりする
パターンを	画像認識等が
検出抽出したりする	ある
画像認識等が	命名された
ある	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
記号処理を	知能の
用いた	記述を
知能の	する
記述を	する
主体と	研究での
する	研究での
情報処理や	アプローチという
研究での	意味あいでも
アプローチという	使われている
意味あいでも	ある
使われている	制御システムや
家庭用電気機械器具の	思考ルーチンも
制御システムや	思考ルーチンも
ゲームソフトの	呼ば

In [48]:
# 3
for sentence in sentences:
    for chunk in sentence.chunks:
        if chunk.dst is not None:
            src_pos = [morph.pos for morph in chunk.morphs]
            dst_pos = [morph.pos for morph in sentence.chunks[chunk.dst].morphs]
            if "名詞" in src_pos and "動詞" in dst_pos:
                src = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
                dst = "".join([morph.surface if morph.pos != "記号" else "" for morph in sentence.chunks[chunk.dst].morphs])
                print(src, dst, sep="\t")

道具を	研究する
問題解決などの	代わって
知的行動を	代わって
人間に	行わせる
実現に関する	される
研究分野とも	される
日本大百科全書(ニッポニカ)』の	述べている
情報工学者通信工学者の	述べている
佐藤理史は	述べている
次のように	述べている
人間の	実現する
知的能力を	実現する
様々な	ある
技術ソフトウェアコンピュータシステム	ある
専門家の	模倣する
エキスパートシステム	解析して
画像データを	検出抽出したりする
特定の	検出抽出したりする
検出抽出したりする	ある
画像認識等が	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	使われている
命名された	使われている
現在では	用いた
知能の	する
記述を	する
アプローチという	使われている
意味あいでも	ある
ゲームソフトの	呼ばれる
思考ルーチンも	呼ばれる
ことも	ある
模倣した	出されるが
プログラム	出されるが
人工無脳が	出されるが
引き合いに	困難視されている
専門家の	させようという
研究情報処理システムの	困難視されている
実現は	持つ
人間が	持つ
常識の	なり
記述が	なり
問題と	困難視されている
実用への	困難視されている
利用が	ある
実現への	知られているが
ニューラルネットワークなどのような	知られているが
アプローチも	ある
(GoodOldFashionedAI)との	ある
記述の	ある
記号的明示性に	集めた
その後	集めた
サポートベクターマシンが	集めた
注目を	ある
自らの	行う
経験を	行う
元に	行う
強化学習という	ある
レイカーツワイルという	実装するという
言葉通り	表現し
知性を	表現し
機械的に	実装するという
ビッグデータの	行った
一過性の	超えて
流行を	浸透して
社会に	行った
浸透して	なった
2016年から	導入した
2017年にかけて	導入した
世界トップクラスの	破り
プレイヤーも	なった
麻雀では	到達するなど
MicrosoftSuphx(SuperPhoenix)」が	到達するなど
AIとして	到達するなど
十段に	なった
時代の	なった
最先端技術と	なった
第２次人工知能ブームでの	呼ばれ
人工知能は	呼ばれ
機械学習と	ある
以下のような	ある
ものが	ある
開

In [55]:
# 4
import graphviz
from IPython.display import Image, display_png

g = graphviz.Digraph(format="png")

sentence = sentences[7]
for chunk in sentence.chunks:
    if chunk.dst is not None:
        src = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
        dst = "".join([morph.surface if morph.pos != "記号" else "" for morph in sentence.chunks[chunk.dst].morphs])
        g.edge(src, dst)


g.view()

'Digraph.gv.png'

In [69]:
# 5
with open("chapter5-5.txt", "w") as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            for morph in chunk.morphs:
                if morph.pos == "動詞":
                    ans = []
                    for src in chunk.srcs:
                        ans += [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == "助詞"]
                    if len(ans) > 0:
                        ans = " ".join(sorted(list(set(ans))))
                        line = f"{morph.base}\t{ans}\n"
                        f.writelines(line)
                    break

In [70]:
!cat chapter5-5.txt | sort | uniq -c | sort -nr | head -n 10

  31 する	の
  21 する	を
  14 する	が
  12 する	に
  11 する	は
  11 する	の を
   6 する	や
   6 する	に は
   5 する	の は
   5 する	に の


In [72]:
!cat chapter5-5.txt | grep '行う' | sort | uniq -c | sort -nr | head -n 5

   3 行う	を
   2 行う	は
   2 行う	て に
   1 行う	は まで
   1 行う	の


In [73]:
!cat chapter5-5.txt | grep 'なる' | sort | uniq -c | sort -nr | head -n 5

   3 なる	が の
   2 なる	は
   1 異なる	の
   1 異なる	と も
   1 無くなる	として


In [74]:
!cat chapter5-5.txt | grep '与える' | sort | uniq -c | sort -nr | head -n 5

   1 与える	に の や
   1 与える	が や
   1 与える	が に も


In [ ]:
# 6


In [ ]:
# 7


In [ ]:
# 8


In [ ]:
# 9
